# Decorators
A decorator is a function that takes a function as a parameter and returns a function. In other words, a decorator wraps a function and modify its behavior without modifying the function itself. This is possible because functions are **first class objects** in Python, this means that:
- We can store a function in a variable
- We can pass a function as a paremeter to another function
- We can return a function from a function
- We can store a function in data structures as lists or dictionaries
Decorators can  be added to functions and classes

The basic structure of a decorator is the following:

In [1]:
import time

def timer(func):  # We pass a function as a paremeter
    def wrapper():
        begin = time.perf_counter()
        func()
        end = time.perf_counter()
        print(func.__name__, "takes", end - begin)
    return wrapper  # We return a function from the timer function

@timer
def greet():
    print("Hello World!")

def greet_copy():
    print("Hello World!")
    
greet()

# Decorator usage is equivalent to the following
greet_copy = timer(greet_copy)
greet_copy()

Hello World!
greet takes 8.710700058145449e-05
Hello World!
greet_copy takes 9.997998859034851e-06


In order to pass arguments and kwarguments from the function in the decorator we use *args and /*/*kwargs. Furthermore, we can store and return the function value to do something after it's called.

In [4]:
import time

def timer(func):  # We pass a function as a paremeter
    def wrapper(*args, **kwargs):  # We get the *args and **kwargs in the wrapper and pass it to the func
        begin = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(func.__name__, "takes", end - begin, "with args =", args, "kwargs =", kwargs)
        return result
    return wrapper  # We return a function from the timer function

@timer
def sum_nums(*args):
    return sum(args)
@timer
def greet_by_name(name: str):
    print("Hello", name, "!")

print(sum_nums(2, 3, 4))
# We can pass args or kwargs
greet_by_name("Juan")
greet_by_name(name="Sebastian")

sum_nums takes 2.2709973563905805e-06 with args = (2, 3, 4) kwargs = {}
9
Hello Juan !
greet_by_name takes 1.826300285756588e-05 with args = ('Juan',) kwargs = {}
Hello Sebastian !
greet_by_name takes 1.928100027726032e-05 with args = () kwargs = {'name': 'Sebastian'}


# Keeping function properties
When a decorator is passed to a function it returns the wrapper function

In [13]:
import time

def timer(func):
    print("[timer] setting up decorator...")
    def wrapper(*args, **kwargs):
        begin = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print("[timer]", func.__name__, "takes", end - begin, "with args =", args, "kwargs =", kwargs)
        return result
    return wrapper

@timer
def greet_by_name(name: str):
    """
    This is a function that greets by name
    """
    print("Hello", name, "!")

print(greet_by_name.__name__)
print(greet_by_name.__doc__)

[timer] setting up decorator...
wrapper
None


We can solve this passing the func properties to the wrapper

In [14]:
import time

def timer(func):
    print("[timer] setting up decorator...")
    def wrapper(*args, **kwargs):
        begin = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print("[timer]", func.__name__, "takes", end - begin, "with args =", args, "kwargs =", kwargs)
        return result
    wrapper.__name__ = func.__name__
    wrapper.__doc__ = func.__doc__
    return wrapper

@timer
def greet_by_name(name: str):
    """
    This is a function that greets by name
    """
    print("Hello", name, "!")

print(greet_by_name.__name__)
print(greet_by_name.__doc__)

[timer] setting up decorator...
greet_by_name

    This is a function that greets by name
    


We can obtain the same result using the **functools.wraps** builtin function

In [18]:
import time
from functools import wraps

def timer(func):
    print("[timer] setting up decorator...")
    @wraps(func)
    def wrapper(*args, **kwargs):
        begin = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print("[timer]", func.__name__, "takes", end - begin, "with args =", args, "kwargs =", kwargs)
        return result
    return wrapper

@timer
def greet_by_name(name: str):
    """
    This is a function that greets by name
    """
    print("Hello", name, "!")

print(greet_by_name.__name__)
print(greet_by_name.__doc__)

[timer] setting up decorator...
greet_by_name

    This is a function that greets by name
    


## Nested decorators
We can chain decorators, they will be executed from top to bottom

In [19]:
import time
from functools import wraps

def timer(func):
    print("[timer] setting up decorator...")
    @wraps(func)
    def wrapper(*args, **kwargs):
        begin = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print("[timer]", func.__name__, "takes", end - begin, "with args =", args, "kwargs =", kwargs)
        return result
    return wrapper
    

def show_parameters(func):
    print("[show_parameters] setting up decorator...")
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("[show_parameters]", func.__name__, "called", "with args =", args, "kwargs =", kwargs)
        return func(*args, **kwargs)
    return wrapper

@show_parameters
@timer
def sum_nums(*args):
    return sum(args)

@show_parameters
@timer
def greet_by_name(name: str):
    print("Hello", name, "!")

print("Functions definition done")
print(sum_nums(2, 3, 4))
greet_by_name("Juan")
greet_by_name(name="Sebastian")
print(greet_by_name.__name__)  # functools.wraps works with nested decorators

# the above is equivalent to
def greet_by_name_v2(name: str):
    print("Hello", name, "!")
greet_by_name_v2 = show_parameters(timer(greet_by_name_v2))
greet_by_name_v2("Juan Sebastian")
print(greet_by_name_v2.__name__)

[timer] setting up decorator...
[show_parameters] setting up decorator...
[timer] setting up decorator...
[show_parameters] setting up decorator...
Functions definition done
[show_parameters] sum_nums called with args = (2, 3, 4) kwargs = {}
[timer] sum_nums takes 3.4050026442855597e-06 with args = (2, 3, 4) kwargs = {}
9
[show_parameters] greet_by_name called with args = ('Juan',) kwargs = {}
Hello Juan !
[timer] greet_by_name takes 2.33470018429216e-05 with args = ('Juan',) kwargs = {}
[show_parameters] greet_by_name called with args = () kwargs = {'name': 'Sebastian'}
Hello Sebastian !
[timer] greet_by_name takes 2.5508001272100955e-05 with args = () kwargs = {'name': 'Sebastian'}
greet_by_name
[timer] setting up decorator...
[show_parameters] setting up decorator...
[show_parameters] greet_by_name_v2 called with args = ('Juan Sebastian',) kwargs = {}
Hello Juan Sebastian !
[timer] greet_by_name_v2 takes 2.4770997697487473e-05 with args = ('Juan Sebastian',) kwargs = {}
greet_by_nam

## Decorators with arguments
We can pass arguments to decorators nesting the decorator itself in another function  

In [53]:
import random
from functools import wraps

def repeat(*opt, num: int = 2):
    print("[repeat]")    
    if len(opt) == 1 and callable(opt[0]):  # The decorator was called without params with the function
        print("[repeat_no_arguments]")
        @wraps(opt[0])
        def wrapper(*args, **kwargs):
            print("[wrapper]")
            return [opt[0](*args, **kwargs) for _ in range(num)]
        return wrapper
    
    def repeat_func(func):
        print("[repeat_arguments]")
        print("[repeat_func]")
        @wraps(func)
        def wrapper(*args, **kwargs):
            print("[repeat_func: wrapper]")
            return [func(*args, **kwargs) for _ in range(num if not opt else opt[0])]
        return wrapper
    return repeat_func


@repeat(6)
def get_decimal_random(low: int = 0, high: int = 10):
    return random.randint(low, high)

@repeat
def get_decimal_random_2(low: int = 0, high: int = 10):
    return random.randint(low, high)

@repeat(num=6)
def get_decimal_random_3(low: int = 0, high: int = 10):
    return random.randint(low, high)

print("Function configuration done")
print(get_decimal_random())
print(get_decimal_random_2(2, 3))
print(get_decimal_random_3(high=8))
print(get_decimal_random.__name__)  # functools.wraps works with parameterized decorators
print(get_decimal_random_2.__name__)
print(get_decimal_random_3.__name__)

# the above is equivalent to
def get_decimal_random_v2(low: int = 0, high: int = 10):
    return random.randint(low, high)
get_decimal_random_v2 = repeat(6)(get_decimal_random_v2)
print(get_decimal_random_v2())
print(greet_by_name_v2.__name__)

[repeat]
[repeat_arguments]
[repeat_func]
[repeat]
[repeat_no_arguments]
[repeat]
[repeat_arguments]
[repeat_func]
Function configuration done
[repeat_func: wrapper]
[5, 6, 8, 2, 7, 2]
[wrapper]
[2, 3]
[repeat_func: wrapper]
[0, 2, 5, 4, 3, 1]
get_decimal_random
get_decimal_random_2
get_decimal_random_3
[repeat]
[repeat_arguments]
[repeat_func]
[repeat_func: wrapper]
[0, 3, 6, 9, 8, 7]
greet_by_name_v2


## Class as a decorator
As we see until this point, a decorator is a callable. It is more common to use a function, but the classes are callable as well. Then, we can define a decorator with a function.

In [28]:
import time
from functools import update_wrapper

class Timer:
    def __init__(self, func):
        print("[Timer] setting up decorator...")
        self.func = func
        update_wrapper(self, func)  # We can pass the func properties to the class with update_wrapper from functools
    
    def __call__(self, *args, **kwargs):
        begin = time.perf_counter()
        result = self.func(*args, **kwargs)
        end = time.perf_counter()
        print("[Timer]", self.func.__name__, "takes", end - begin, "with args =", args, "kwargs =", kwargs)
        return result

@Timer
def greet_by_name(name: str):
    print("Hello", name, "!")

greet_by_name("Juan")
greet_by_name(name="Sebastian")
print(greet_by_name.__name__)

[Timer] setting up decorator...
Hello Juan !
[Timer] greet_by_name takes 3.0414998036576435e-05 with args = ('Juan',) kwargs = {}
Hello Sebastian !
[Timer] greet_by_name takes 2.931500057457015e-05 with args = () kwargs = {'name': 'Sebastian'}
greet_by_name


To create a decorator with arguents using a class we need to use a helper function

In [36]:
import time
from functools import update_wrapper, wraps

class _Timer:
    def __init__(self, func = None, prefix: str = None, suffix: str = None):
        print("[Timer] setting up decorator...")
        self.func = func
        self.prefix = prefix
        self.suffix = suffix
        update_wrapper(self, func)  # We can pass the func properties to the class with update_wrapper from functools
    
    def __call__(self, *args, **kwargs):
        begin = time.perf_counter()
        result = self.func(*args, **kwargs)
        end = time.perf_counter()
        print(self.prefix, self.func.__name__, "takes", end - begin, "with args =", args, "kwargs =", kwargs, self.suffix)
        return result

def Timer(func = None, prefix: str = None, suffix: str = None):
    if func:
        return _Timer(func)
    else:
        @wraps(func)
        def wrapper(func):
            return _Timer(func, prefix, suffix)
        return wrapper

@Timer(prefix="[Timer]")  # We need to specify the param, if not it takes the input as the func param
def greet_by_name(name: str):
    print("Hello", name, "!")

@Timer
def greet_by_name_2(name: str):
    print("Hello", name, "!")

greet_by_name("Juan")
greet_by_name_2(name="Sebastian")
print(greet_by_name.__name__)
print(greet_by_name_2.__name__)

[Timer] setting up decorator...
[Timer] setting up decorator...
Hello Juan !
[Timer] greet_by_name takes 2.841099922079593e-05 with args = ('Juan',) kwargs = {} None
Hello Sebastian !
None greet_by_name_2 takes 2.763399970717728e-05 with args = () kwargs = {'name': 'Sebastian'} None
greet_by_name
greet_by_name_2


## Decorating classes
Decorating a class is simmilar to decorate a function, but we return the actual class

In [47]:
def logmethods(cls):
    for key, value in vars(cls).items():
        print(key, value)
    return cls  # we return a class not a function

def repeat(num: int = 2):
    print("[repeat]")
    def repeat_func(func):
        print("[repeat_func]")
        @wraps(func)
        def wrapper(*args, **kwargs):  # We are passing the default self argument inside a method class
            print("[wrapper]")
            return [func(*args, **kwargs) for _ in range(num)]
        return wrapper
    return repeat_func

@logmethods
class Spam:
    def __init__(self, value) -> None:
        self.value = value
    
    @repeat(2)
    def grow(self):
        print("Grow!")

spam = Spam("Hello")
spam.grow()
print(Spam)

[repeat]
[repeat_func]
__module__ __main__
__init__ <function Spam.__init__ at 0x7f94f23a9990>
grow <function Spam.grow at 0x7f94f23a9c60>
__dict__ <attribute '__dict__' of 'Spam' objects>
__weakref__ <attribute '__weakref__' of 'Spam' objects>
__doc__ None
[wrapper]
Grow!
Grow!
<class '__main__.Spam'>
